In [3]:
# import requests
# import pandas as pd
# import xml.etree.ElementTree as ET
# import time
# import os

# # 1) 주소 → 위경도 변환
# def get_coords_from_vworld(address, api_key):
#     url = "https://api.vworld.kr/req/address"
#     params = {
#         'service': 'address',
#         'version': '2.0',
#         'request': 'GetCoord',
#         'format': 'json',
#         'key': api_key,
#         'type': 'ROAD',   # ROAD / PARCEL / BOTH
#         'address': address,
#         'crs': 'EPSG:4326'
#     }
#     res = requests.get(url, params=params, timeout=10)
#     res.raise_for_status()
#     resp = res.json().get('response', {})
#     if resp.get('status') != 'OK':
#         raise RuntimeError(f"VWorld 주소 변환 오류: {resp.get('error')}")
#     result = resp['result']
#     items = result.get('items') or result.get('item')
#     if items:
#         feat = items[0] if isinstance(items, list) else items
#         point = feat['point']
#     else:
#         point = result.get('point')
#     return float(point['x']), float(point['y'])


# # 2) 위경도로 배수등급 조회 & λ 매핑
# def fetch_soil_drainage_with_lambda(lon, lat, data_api_key):
#     # 2-1) 원격 호출
#     url = 'https://api.vworld.kr/req/data'
#     params = {
#         'key': data_api_key,
#         'service': 'data',
#         'request': 'GetFeature',
#         'data': 'LT_C_ASITSOILDRA',
#         'format': 'json',
#         'geometry': 'true',
#         'page': '1',
#         'size': '1000',
#         'crs': 'EPSG:4326',
#         'domain': 'localhost',
#         'geomFilter': f'POINT({lon} {lat})'
#     }
#     res = requests.get(url, params=params, timeout=10)
#     res.raise_for_status()
#     feats = res.json()['response']['result']['featureCollection']['features']
    
#     # 2-2) DataFrame 변환
#     soil_df = pd.DataFrame([f['properties'] for f in feats])
    
#     # 2-3) 등급 → λ 매핑 테이블
#     drain_to_lambda = {
#         '매우양호': 0.50,
#         '양호':     0.35,
#         '약간양호': 0.25,
#         '약간불량': 0.15,
#         '불량':     0.10,
#         '매우불량': 0.05,
#     }
    
#     # 2-4) 'label' 컬럼 기준으로 λ 추가
#     soil_df['lambda'] = soil_df['label'].map(drain_to_lambda)
#     return soil_df[['label', 'lambda']]


# if __name__ == '__main__':
#     # 사용자 입력
#     address      = input("📍 조회할 주소: ")
#     vworld_addr  = 'B4C5E64B-E7B8-3103-9156-289899AE4279'  # 주소 API 키
#     vworld_data  = '5EE19860-B1C7-3082-B027-99245C4FC9BE'  # 데이터 API 키

#     # 주소 → 위경도
#     lon, lat = get_coords_from_vworld(address, vworld_addr)
#     soil_df = fetch_soil_drainage_with_lambda(lon, lat, vworld_data)
#     print(f"▶ 변환 좌표: {address} -> 경도={lon:.6f}, 위도={lat:.6f}")

#     # 배수등급 조회 + λ 매핑
#     result = fetch_soil_drainage_with_lambda(lon, lat, vworld_data)
#     print("✅ 배수등급 ↔ 손실계수 λ")
#     print(result.to_string(index=False))
    

    
# #------
# service_key = 'OaE7WFXPyKXCPtSvtE9HuQdSwbzhl/C9FhjkxVzyOfKLRZxqAMChtLhArevfCux2XuluPYLtgDuMUEPXvGaoNQ=='
# url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'

# df_station = pd.read_csv('csv/ASOS_station.csv')
# station_dict = dict(zip(df_station['지점명'], df_station['지점']))

# region = input("📍 조회할 지점명을 입력하세요 (예: 서울, 수원, 대전 등): ").strip()
# stn_id = str(station_dict.get(region, '108'))  # 기본값: 서울

# print(f"선택된 지점명: {region} → 지점 코드: {stn_id}")

# params ={'serviceKey' : service_key, 
#          'pageNo' : '1', 
#          'numOfRows' : '100', 
#          'dataType' : 'XML', 
#          'dataCd' : 'ASOS', 
#          'dateCd' : 'DAY', 
#          'startDt' : '20240301', 
#          'endDt' : '20240901', 
#          'stnIds' : stn_id }



# all_data = []
# page = 1

# while True:
#     print(f'Fetching page {page}...')
#     params = params.copy()
#     params['pageNo'] = str(page)
    
#     response = requests.get(url, params=params)
#     root = ET.fromstring(response.content)

#     print(response.content)

#     items = root.findall('.//item')
#     if not items:
#         print('✅ All data fetched.')
#         break
    
#     for item in items:
#         all_data.append({
#             'date': item.findtext('tm'),
#             'avg_temp': item.findtext('avgTa'),
#             'min_temp': item.findtext('minTa'),
#             'max_temp': item.findtext('maxTa'),
#             'rainfall': item.findtext('sumRn'),
#             'humidity': item.findtext('avgRhm'),
#             'sunshine': item.findtext('sumGsr'),
#             'wind': item.findtext('avgWs'),
#             'air_pressure': item.findtext('avgPa')
#         })
    
#     page += 1
#     time.sleep(0.3)

# # DataFrame 생성
# df = pd.DataFrame(all_data)

# df['date'] = pd.to_datetime(df['date'])
# df = df.sort_values('date').reset_index(drop=True)

# for col in ['avg_temp', 'min_temp', 'max_temp', 'rainfall', 'humidity', 'sunshine', 'wind', 'air_pressure']:
#     df[col] = pd.to_numeric(df[col], errors='coerce')

# df['rainfall'].fillna(0, inplace = True)

# # 인덱스를 파일에 포함시키지 않을 때
# os.makedirs('results', exist_ok=True)
# output_path = os.path.join('results', 'weather.csv')
# df.to_csv(output_path, index=False, encoding='utf-8-sig')
# print(f'\n✔ Saved weather results to {output_path}')



# # --- 위에서 정의한 함수 --- 
# def calculate_irrigation(df, 
#                          lam,
#                          kc=1.15,# 배수 손실계수 λ
#                          init_res=0   # t=0일 전 잔존수분
#                         ):
#     df = df.copy()
#     df['date'] = pd.to_datetime(df['date'])
#     for col in ['avg_temp','min_temp','max_temp','rainfall','sunshine']:
#         df[col] = pd.to_numeric(df[col], errors='coerce')
#     df['rainfall'].fillna(0, inplace=True)

#     def _eto(r):
#         if pd.notna(r['avg_temp']) and pd.notna(r['min_temp']) \
#            and pd.notna(r['max_temp']) and pd.notna(r['sunshine']):
#             return 0.0023 * (r['avg_temp']+17.8) \
#                    * ((r['max_temp']-r['min_temp'])**0.5) * r['sunshine']
#         return 0
#     df['ETo'] = df.apply(_eto, axis=1)
#     df['ETc'] = df['ETo'] * kc
#     df['Pe']  = df['rainfall'] * 0.8

#     df['irrigation'] = 0.0
#     df['residual']   = 0.0

#     res_prev = init_res

#     for idx, row in df.iterrows():
#         P  = row['Pe']
#         ET = row['ETc']

#         I = max(0, ET - (P + res_prev) + lam*(P + res_prev))
#         res_t = max(0, res_prev + P + I - ET - (1-lam)*(P + I))

#         df.at[idx, 'irrigation'] = I
#         df.at[idx, 'residual']   = res_t

#         res_prev = res_t

#     return df[['date','avg_temp','min_temp','max_temp',
#                'rainfall','ETo','ETc','Pe','irrigation','residual']]


# # --- 스크립트 메인 흐름 ---
# if __name__ == '__main__':
#     # 1) 입력: weather.csv 읽기
#     df_weather = pd.read_csv('results/weather.csv')   # 실제 파일명에 맞게 수정

#     # 2) 관개량/잔존수분 계산
#     df_result = calculate_irrigation(
#         df_weather,
#         kc=1.15,    # 작물별 Kc
#         lam = soil_df['lambda'].iloc[0],   # 배수 손실계수 λ
#         init_res=0  # 초기 잔존수분
#     )


#     # 3) 결과를 CSV로 저장
#     output_path = os.path.join('results', 'irrigation_results.csv')
#     df_result.to_csv(output_path, index=False, encoding='utf-8-sig')
#     print(f'\n✔ Saved results to {output_path}')

# df_result

▶ 변환 좌표: 서울특별시 서초구 서초중앙로 200 -> 경도=127.017017, 위도=37.499203
✅ 배수등급 ↔ 손실계수 λ
label  lambda
   양호    0.35
선택된 지점명: 서울 → 지점 코드: 108
Fetching page 1...
b'<?xml version="1.0" encoding="UTF-8"?>\r\n<response><header><resultCode>00</resultCode><resultMsg>NORMAL_SERVICE</resultMsg></header><body><dataType>XML</dataType><items><item><stnId>108</stnId><stnNm>\xec\x84\x9c\xec\x9a\xb8</stnNm><tm>2024-03-01</tm><avgTa>-4.1</avgTa><minTa>-7.1</minTa><minTaHrmt>2355</minTaHrmt><maxTa>0.3</maxTa><maxTaHrmt>0001</maxTaHrmt><mi10MaxRn></mi10MaxRn><mi10MaxRnHrmt></mi10MaxRnHrmt><hr1MaxRn></hr1MaxRn><hr1MaxRnHrmt></hr1MaxRnHrmt><sumRnDur></sumRnDur><sumRn></sumRn><maxInsWs>14.2</maxInsWs><maxInsWsWd>270</maxInsWsWd><maxInsWsHrmt>1427</maxInsWsHrmt><maxWs>7.5</maxWs><maxWsWd>290</maxWsWd><maxWsHrmt>1434</maxWsHrmt><avgWs>4.4</avgWs><hr24SumRws>3783</hr24SumRws><maxWd>290</maxWd><avgTd>-16.3</avgTd><minRhm>24</minRhm><minRhmHrmt>1359</minRhmHrmt><avgRhm>38.9</avgRhm><avgPv>1.8</avgPv><avgPa>1013.3</avgPa><m

/var/folders/j4/2041ftmn23q6ckn4xskb5z9c0000gn/T/ipykernel_21120/3237006637.py:159: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['rainfall'].fillna(0, inplace = True)
/var/folders/j4/2041ftmn23q6ckn4xskb5z9c0000gn/T/ipykernel_21120/3237006637.py:179: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b

,date,avg_temp,min_temp,max_temp,rainfall,ETo,ETc,Pe,irrigation,residual
0,2024-03-01,-4.1,-7.1,0.3,0.0,1.742616,2.004008,0.0,2.004008,0.0
1,2024-03-02,-0.5,-8.1,4.1,0.0,1.835932,2.111322,0.0,2.111322,0.0
2,2024-03-03,4.0,0.4,7.8,0.0,1.844068,2.120678,0.0,2.120678,0.0
3,2024-03-04,5.4,-1.8,11.7,0.0,3.487857,4.011035,0.0,4.011035,0.0
4,2024-03-05,8.0,5.5,11.0,0.0,0.875346,1.006647,0.0,1.006647,0.0
...,...,...,...,...,...,...,...,...,...,...
180,2024-08-28,28.2,23.0,33.2,0.0,8.102797,9.318216,0.0,9.318216,0.0
181,2024-08-29,28.4,23.4,33.5,0.0,8.395211,9.654493,0.0,9.654493,0.0
182,2024-08-30,28.0,23.7,32.9,0.0,7.837630,9.013275,0.0,9.013275,0.0
183,2024-08-31,27.6,22.6,33.0,0.0,8.132377,9.352233,0.0,9.352233,0.0


In [17]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import time
import os
from datetime import datetime, timedelta

file_path = 'csv/작물별_Kc.csv'

# 엑셀 불러오기
df = pd.read_csv(file_path)


# 1. 사용자 입력
start_date = input("재배 시작 날짜 (예: 2024-04-15): ")
today = input("오늘 날짜 입력 (예: 2024-05-20): ")
crop = input("작물명 입력 (예: 당근): ")

# 2. 날짜 차이 계산
start_dt = datetime.strptime(start_date, "%Y-%m-%d")
today_dt = datetime.strptime(today, "%Y-%m-%d")
days_passed = abs((today_dt - start_dt).days)
print(f"경과일수: {days_passed}일")

start_dt_str = start_date.replace("-", "")
end_dt_str = today.replace("-", "")

# 3. 작물 정보 불러오기
crop_row = df[df["작물 종류"] == crop].iloc[0]
total_days = crop_row["생육일수"]

# 4. 단계 구간 계산
ini = int(total_days * 0.2)
mid = int(total_days * 0.6)
end = total_days - ini - mid

# 5. 현재 단계 판단
if days_passed <= ini:
    stage = "ini"
    kc = crop_row["Kc_ini"]
elif days_passed <= ini + mid:
    stage = "mid"
    kc = crop_row["Kc_mid"]
else:
    stage = "end"
    kc = crop_row["Kc_end"]


# 1) 주소 → 위경도 변환
def get_coords_from_vworld(address, api_key):
    url = "https://api.vworld.kr/req/address"
    params = {
        'service': 'address',
        'version': '2.0',
        'request': 'GetCoord',
        'format': 'json',
        'key': api_key,
        'type': 'ROAD',   # ROAD / PARCEL / BOTH
        'address': address,
        'crs': 'EPSG:4326'
    }
    res = requests.get(url, params=params, timeout=10)
    res.raise_for_status()
    resp = res.json().get('response', {})
    if resp.get('status') != 'OK':
        raise RuntimeError(f"VWorld 주소 변환 오류: {resp.get('error')}")
    result = resp['result']
    items = result.get('items') or result.get('item')
    if items:
        feat = items[0] if isinstance(items, list) else items
        point = feat['point']
    else:
        point = result.get('point')
    return float(point['x']), float(point['y'])


# 2) 위경도로 배수등급 조회 & λ 매핑
def fetch_soil_drainage_with_lambda(lon, lat, data_api_key):
    # 2-1) 원격 호출
    url = 'https://api.vworld.kr/req/data'
    params = {
        'key': data_api_key,
        'service': 'data',
        'request': 'GetFeature',
        'data': 'LT_C_ASITSOILDRA',
        'format': 'json',
        'geometry': 'true',
        'page': '1',
        'size': '1000',
        'crs': 'EPSG:4326',
        'domain': 'localhost',
        'geomFilter': f'POINT({lon} {lat})'
    }
    res = requests.get(url, params=params, timeout=10)
    res.raise_for_status()
    feats = res.json()['response']['result']['featureCollection']['features']
    
    # 2-2) DataFrame 변환
    soil_df = pd.DataFrame([f['properties'] for f in feats])
    
    # 2-3) 등급 → λ 매핑 테이블
    drain_to_lambda = {
        '매우양호': 0.50,
        '양호':     0.35,
        '약간양호': 0.25,
        '약간불량': 0.15,
        '불량':     0.10,
        '매우불량': 0.05,
    }
    
    # 2-4) 'label' 컬럼 기준으로 λ 추가
    soil_df['lambda'] = soil_df['label'].map(drain_to_lambda)
    return soil_df[['label', 'lambda']]


if __name__ == '__main__':
    # 사용자 입력
    address      = input("📍 조회할 주소: ")
    vworld_addr  = 'B4C5E64B-E7B8-3103-9156-289899AE4279'  # 주소 API 키
    vworld_data  = '5EE19860-B1C7-3082-B027-99245C4FC9BE'  # 데이터 API 키

    # 주소 → 위경도
    lon, lat = get_coords_from_vworld(address, vworld_addr)
    soil_df = fetch_soil_drainage_with_lambda(lon, lat, vworld_data)
    print(f"▶ 변환 좌표: {address} -> 경도={lon:.6f}, 위도={lat:.6f}")

    # 배수등급 조회 + λ 매핑
    result = fetch_soil_drainage_with_lambda(lon, lat, vworld_data)
    print("✅ 배수등급 ↔ 손실계수 λ")
    print(result.to_string(index=False))
    

    
#------
service_key = 'OaE7WFXPyKXCPtSvtE9HuQdSwbzhl/C9FhjkxVzyOfKLRZxqAMChtLhArevfCux2XuluPYLtgDuMUEPXvGaoNQ=='
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'

df_station = pd.read_csv('csv/ASOS_station.csv')
station_dict = dict(zip(df_station['지점명'], df_station['지점']))

region = input("📍 조회할 지점명을 입력하세요 (예: 서울, 수원, 대전 등): ").strip()
stn_id = str(station_dict.get(region, '108'))  # 기본값: 서울

print(f"선택된 지점명: {region} → 지점 코드: {stn_id}")

params ={'serviceKey' : service_key, 
         'pageNo' : '1', 
         'numOfRows' : '100', 
         'dataType' : 'XML', 
         'dataCd' : 'ASOS', 
         'dateCd' : 'DAY', 
         'startDt' : start_dt_str, 
         'endDt' : end_dt_str, 
         'stnIds' : stn_id }



# all_data = []
# page = 1

# while True:
#     print(f'Fetching page {page}...')
#     params = params.copy()
#     params['pageNo'] = str(page)
    
#     response = requests.get(url, params=params)
#     root = ET.fromstring(response.content)

#     print(response.content)

#     items = root.findall('.//item')
#     if not items:
#         print('✅ All data fetched.')
#         break
    
#     for item in items:
#         all_data.append({
#             'date': item.findtext('tm'),
#             'avg_temp': item.findtext('avgTa'),
#             'min_temp': item.findtext('minTa'),
#             'max_temp': item.findtext('maxTa'),
#             'rainfall': item.findtext('sumRn'),
#             'humidity': item.findtext('avgRhm'),
#             'sunshine': item.findtext('sumGsr'),
#             'wind': item.findtext('avgWs'),
#             'air_pressure': item.findtext('avgPa')
#         })
    
#     page += 1
#     time.sleep(0.3)

all_data = []
page = 1

while True:
    print(f'Fetching page {page}...')
    # params 복사
    this_params = params.copy()
    this_params['pageNo'] = str(page)

    response = requests.get(url, params=this_params)
    root = ET.fromstring(response.content)

    # 항목 추출
    items = root.findall('.//item')
    if not items:
        print('✅ All data fetched (no more items).')
        break

    # 데이터 수집
    for item in items:
        all_data.append({
            'date': item.findtext('tm'),
            'avg_temp': item.findtext('avgTa'),
            'min_temp': item.findtext('minTa'),
            'max_temp': item.findtext('maxTa'),
            'rainfall': item.findtext('sumRn'),
            'humidity': item.findtext('avgRhm'),
            'sunshine': item.findtext('sumGsr'),
            'wind': item.findtext('avgWs'),
            'air_pressure': item.findtext('avgPa')
        })

    # 페이징 정보로 종료 여부 판단
    total_count  = int(root.findtext('.//totalCount'))
    num_of_rows  = int(root.findtext('.//numOfRows'))
    current_page = int(root.findtext('.//pageNo'))
    max_page     = (total_count + num_of_rows - 1) // num_of_rows

    if current_page >= max_page:
        print('✅ All data fetched (last page reached).')
        break

    page += 1
    time.sleep(0.3)

# -----------------------
# DataFrame 생성 및 방어 로직
if not all_data:
    raise RuntimeError("❌ 기상 데이터가 한 건도 수집되지 않았습니다. 날짜 범위나 지점 코드를 확인하세요.")


# DataFrame 생성
df = pd.DataFrame(all_data)

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

for col in ['avg_temp', 'min_temp', 'max_temp', 'rainfall', 'humidity', 'sunshine', 'wind', 'air_pressure']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['rainfall'].fillna(0, inplace = True)

# 인덱스를 파일에 포함시키지 않을 때
os.makedirs('results', exist_ok=True)
output_path = os.path.join('results', 'weather.csv')
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f'\n✔ Saved weather results to {output_path}')



# --- 위에서 정의한 함수 --- 
def calculate_irrigation(df, 
                         lam,
                         kc,# 배수 손실계수 λ
                         init_res=0   # t=0일 전 잔존수분
                        ):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    for col in ['avg_temp','min_temp','max_temp','rainfall','sunshine']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df['rainfall'].fillna(0, inplace=True)

    def _eto(r):
        if pd.notna(r['avg_temp']) and pd.notna(r['min_temp']) \
           and pd.notna(r['max_temp']) and pd.notna(r['sunshine']):
            return 0.0023 * (r['avg_temp']+17.8) \
                   * ((r['max_temp']-r['min_temp'])**0.5) * r['sunshine']
        return 0
    df['ETo'] = df.apply(_eto, axis=1)
    df['ETc'] = df['ETo'] * kc
    df['Pe']  = df['rainfall'] * 0.8

    df['irrigation'] = 0.0
    df['residual']   = 0.0

    res_prev = init_res

    for idx, row in df.iterrows():
        P  = row['Pe']
        ET = row['ETc']

        I = max(0, ET - (P + res_prev) + lam*(P + res_prev))
        res_t = max(0, res_prev - ET - (1-lam)*(P + I))

        df.at[idx, 'irrigation'] = I
        df.at[idx, 'residual']   = res_t

        res_prev = res_t

    return df[['date','avg_temp','min_temp','max_temp',
               'rainfall','ETo','ETc','Pe','irrigation','residual']]


# --- 스크립트 메인 흐름 ---
if __name__ == '__main__':
    # 1) 입력: weather.csv 읽기
    df_weather = pd.read_csv('results/weather.csv')   # 실제 파일명에 맞게 수정

    # 2) 관개량/잔존수분 계산
    df_result = calculate_irrigation(
        df_weather,
        kc=kc,    # 작물별 Kc
        lam = soil_df['lambda'].iloc[0],   # 배수 손실계수 λ
        init_res=0  # 초기 잔존수분
    )


    # 3) 결과를 CSV로 저장
    output_path = os.path.join('results', 'irrigation_results.csv')
    df_result.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f'\n✔ Saved results to {output_path}')

print(df_result.head())
print(df_result.tail()) 

area       = float(input("밭 면적을 입력 (m^2): "))
last_date  = input("마지막으로 물을 준 날짜 (예: 2024-05-07): ")
last_dt  = datetime.strptime(last_date, "%Y-%m-%d")

next_dt = last_dt + timedelta(days=1)

mask = (df_result['date'] >= next_dt) & (df_result['date'] <= today_dt)
cum_irrigation_per_m2 = df_result.loc[mask, 'irrigation'].sum()

total_water_m3 = cum_irrigation_per_m2 * area

print(next_dt)
print(today_dt)
print(f"\n▶ 지난 급수일 다음날부터 오늘까지 1 m²당 누적 관개량: {cum_irrigation_per_m2:.2f} L/m^2")
print(f"▶ 밭 전체(면적 {area:.1f} m²)에서 필요한 총 물량: {total_water_m3:.3f} L")

경과일수: 35일
▶ 변환 좌표: 서울특별시 서초구 서초중앙로 200 -> 경도=127.017017, 위도=37.499203
✅ 배수등급 ↔ 손실계수 λ
label  lambda
   양호    0.35
선택된 지점명: 서울 → 지점 코드: 108
Fetching page 1...


/var/folders/j4/2041ftmn23q6ckn4xskb5z9c0000gn/T/ipykernel_21120/309513259.py:252: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['rainfall'].fillna(0, inplace = True)
/var/folders/j4/2041ftmn23q6ckn4xskb5z9c0000gn/T/ipykernel_21120/309513259.py:272: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

✅ All data fetched (last page reached).

✔ Saved weather results to results/weather.csv

✔ Saved results to results/irrigation_results.csv
        date  avg_temp  min_temp  max_temp  rainfall       ETo       ETc  \
0 2024-04-15      16.0      11.7      20.0      17.1  0.712214  0.747825   
1 2024-04-16      13.3      11.5      17.9       2.3  2.433887  2.555582   
2 2024-04-17      15.0       9.7      21.8       0.0  6.088108  6.392513   
3 2024-04-18      17.3      10.7      24.9       0.0  7.742245  8.129357   
4 2024-04-19      17.6      11.5      24.3       0.0  6.845481  7.187755   

      Pe  irrigation  residual  
0  13.68    0.000000       0.0  
1   1.84    1.359582       0.0  
2   0.00    6.392513       0.0  
3   0.00    8.129357       0.0  
4   0.00    7.187755       0.0  
         date  avg_temp  min_temp  max_temp  rainfall       ETo       ETc  \
31 2024-05-16      13.7       6.9      19.8       0.1  8.157757  8.565645   
32 2024-05-17      18.3      10.8      24.9       0.